In [1]:
import sys
import numpy
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout,Flatten
from keras.layers import LSTM
from keras.callbacks import ModelCheckpoint
from keras.utils import np_utils

from keras.models import load_model, Model
from keras.layers import Dense, Activation, Dropout, Input, LSTM, Reshape, Lambda, RepeatVector
from keras.initializers import glorot_uniform
from keras.utils import to_categorical
from keras.optimizers import Adam
from keras import backend as K
# load ascii text and covert to lowercase

C:\Users\Sergey\AppData\Local\conda\conda\envs\condathree\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
import pickle
import numpy as np
import keras

In [3]:
from keras.utils import to_categorical

data = pickle.load( open( "jazz123.txt", "rb" ))
lengths = [len(x) for x in data]
idx = np.cumsum(lengths)
d = sum(data,[])
e = to_categorical(d)
data_rec = [e[:idx[0]]] + [e[idx[t]:idx[t+1]] for t in range(len(idx) - 2)]

In [6]:
data_rec[1][0]

array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0.])

Version #1: 49 -> 1 

In [4]:
dataX = []
dataY = []

seq_length = 50

for seq in data_rec: 
    n_chars = len(seq)
    for i in range(0, n_chars - seq_length, 1):
        if (seq[i][0] == 1.0 or seq[i][1] == 1.0 ):
            continue
        seq_in = seq[i:i + seq_length]
        seq_out = seq[i + seq_length]
        dataX.append(seq_in)
        dataY.append(seq_out)
n_patterns = len(dataX)


In [8]:
len(dataX)

1112432

Version #2: 50 целиком

In [5]:
len(dataX[0][0])

128

In [97]:
number = 100000

X = numpy.reshape(dataX[:number], (number, seq_length, 128))
y = np.array(dataY[:number])

In [86]:
number = 10000
for i in range(120):
    
    X = numpy.reshape(dataX[i * number:(i+1) * number], (number, seq_length, 128))
    y = np.array(dataY[i * number:(i+1) * number])
    
    with open("data_X_" + str(i), "wb") as fp:  
        np.save(fp, X, allow_pickle = True, fix_imports=False)
    
    with open("data_y_" + str(i), "wb") as fp:  
        np.save(fp, y, allow_pickle = True, fix_imports=False)
    

KeyboardInterrupt: 

In [6]:
filepath="weights-improvement-{epoch:02d}-{loss:.4f}.hdf5"
filepath

'weights-improvement-{epoch:02d}-{loss:.4f}.hdf5'

In [95]:
model = Sequential()
model.add(LSTM(200, input_shape=(X.shape[1], X.shape[2]), return_sequences=True))
model.add(LSTM(100, input_shape=(X.shape[1], X.shape[2]), return_sequences=True))#added

model.add(Dropout(0.2))

model.add(Flatten())
model.add(Dense(y.shape[1], activation='softmax'))

filepath="2Xmodel-{epoch:02d}-{loss:.3f}.hdf5"
checkpoint = ModelCheckpoint(filepath, monitor='loss', verbose=1, save_best_only = True, mode='min')
callbacks_list = [checkpoint]

opt = keras.optimizers.Adam()
model.compile(loss='categorical_crossentropy', optimizer=opt)


In [7]:
model.load_weights("model-01-1.302.hdf5")

In [96]:
n_epochs = 5
number = 100000

L = [i for i in range(11)]

for epoch in range(n_epochs):
    
    np.random.shuffle(L)
    
    for j, i in enumerate(L):
        
        
        print("downloading data....")
        X = numpy.reshape(dataX[i * number:(i+1) * number], (number, seq_length, 128))
        y = np.array(dataY[i * number:(i+1) * number])
        
        
        print("epoch: %d, part: %d, number: %d" % (epoch, j, i))
        model.fit(X, y, epochs = 1, verbose=1, batch_size = 128, callbacks = callbacks_list, shuffle = True)

downloading data....
epoch: 0, part: 0, number: 7
Epoch 1/1
 56320/100000 [===============>..............] - ETA: 2:13 - loss: 1.9539

KeyboardInterrupt: 

In [86]:
model.save("total_sunday.hdf5")

In [87]:
"%d + %d"%(1,2)

'1 + 2'

In [98]:
model.fit(X, y, epochs = 2, verbose=1, batch_size = 256)

Epoch 1/2
100000/100000 [==============================] - 261s 3ms/step - loss: 1.7632
Epoch 2/2
 71168/100000 [====================>.........] - ETA: 1:15 - loss: 1.6806

KeyboardInterrupt: 

In [31]:
X_in = X[0].reshape((1, 50, 128))

In [35]:
model.predict(X_in)

array([[3.02089757e-04, 7.29699731e-01, 4.81484392e-07, 5.07727975e-07,
        4.65268670e-07, 4.97117639e-07, 5.17919318e-07, 4.38293711e-07,
        4.94290703e-07, 5.40053734e-05, 4.33493227e-07, 4.24668684e-07,
        4.55670772e-07, 5.74965270e-07, 4.84270458e-07, 4.57068722e-07,
        5.48520063e-07, 4.43684343e-07, 4.98198290e-07, 4.62957814e-07,
        4.50848489e-07, 1.07666267e-06, 5.57208296e-06, 2.07643416e-06,
        6.50653919e-06, 6.11844598e-06, 1.63248351e-05, 2.08976446e-04,
        6.75121919e-05, 1.13725034e-03, 2.89812335e-04, 2.44551688e-03,
        9.87526611e-04, 7.16817472e-03, 5.92686329e-03, 5.87560795e-03,
        5.66303264e-03, 3.21254390e-03, 3.52350287e-02, 5.68942493e-03,
        7.39225559e-03, 3.86381447e-02, 1.42651238e-03, 7.06781447e-02,
        6.86901622e-03, 3.45782414e-02, 1.13773420e-02, 2.76215211e-03,
        1.95123709e-03, 2.37547420e-03, 6.32626330e-03, 6.03665481e-04,
        1.73663010e-03, 8.64834583e-04, 9.80679106e-05, 7.728250

In [85]:
np.random.choice([1,2,3,4,5,6,7,8,9,10])

1

In [9]:
def take_one(pred):
    a = np.log(pred[0]) / 1.0 
    dist = np.exp(a)/np.sum(np.exp(a)) 
    choices = range(len(a))
    return np.random.choice(choices, p=dist)

In [101]:
ch = range(1000000)

X_in = np.reshape(dataX[np.random.choice(ch)] ,(1, 50, 128))

new_track = [np.argmax(x) for x in X_in[0]] + [120, 1, 1, 1]
p = 2
previous = 1
for i in range(600):
    prediction = model.predict(X_in)
    if p!= 1:
        prediction[0,p] /= 10.0 
    else:
        prediction[0,1] *= 2.0
    prediction[0,previous] = 0.0
    p = take_one(prediction)
    if (p != 1 and p != 0):
        previous = p
    #print(p)
    #previous_note = p
    new_track.append(p)
    X_in = list(X_in[0])[1:]
    p_cat = to_categorical(p, num_classes = 128)
    
    X_in.append(p_cat)
    
    X_in = np.reshape(X_in,(1, 50, 128))
    

C:\Users\Sergey\AppData\Local\conda\conda\envs\condathree\lib\site-packages\ipykernel\__main__.py:2: RuntimeWarning: divide by zero encountered in log
  from ipykernel import kernelapp as app


1112432

In [102]:
create_track_new(new_track)

0


In [27]:
from midiutil import MIDIFile

def create_track_new(seq):
    
    grain = 16
    one_tact = 32
    tacts_per_min = 75
    tempo    = one_tact * tacts_per_min  # In BPM
    volume   = 100  # 0-127, as per the MIDI standard
    track    = 0
    channel  = 0
    time     = 0# to learn
    
    MyMIDI = MIDIFile(1)
    MyMIDI.addTempo(track, time, tempo)
    
    
    i = 0
    tick_count = 0
    while(i < len(seq)):
        
        if i == len(seq) - 1:
            MyMIDI.addNote(track, channel, seq[i], tick_count, one_tact / grain, volume)
            tick_count = tick_count + one_tact / grain
            i = i + 1
        else:
            if seq[i + 1] != 1:
                MyMIDI.addNote(track, channel, seq[i], tick_count, one_tact / grain, volume)
                tick_count = tick_count + one_tact / grain 
                i = i + 1
            else:
                i_prev = i
                i = i + 1
                while(seq[i] == 1 and i < len(seq) - 1):
                    i += 1
                
                    
                #i = i + 1#??

                d = i - i_prev
                
                MyMIDI.addNote(track, channel, seq[i_prev], tick_count, d * one_tact / grain, volume)
                tick_count = tick_count + d * one_tact / grain 
                
                if (i == len(seq)):
                        break
                
                if (seq[i] == 0):
                    i_prev = i
                    if (i == len(seq) - 1):
                        break
                    while(seq[i] == 1 and i < len(seq) - 1):
                        i = i + 1
                        tick_count = tick_count + one_tact / grain 
                        

    
    print(track)

    with open("zzz.mid", "wb") as output_file:
        MyMIDI.writeFile(output_file) 

In [23]:
p = take_one(model.predict(X_in))

In [62]:
with open("my_impr-0306.txt", "wb") as fp:   #Pickling
    pickle.dump(new_track, fp)



In [44]:
np.argmax(to_categorical(p, num_classes = 128))

43

In [21]:
encoded[4]

array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0.])

In [22]:
len(d)

524002

In [23]:
import tensorflow